In [1]:
cd ..

C:\Users\pi96doc\Documents\Programming\PythonScripts\StateModeling


In [2]:
# %matplotlib notebook
# %matplotlib inline
#import mpld3
#mpld3.enable_notebook()

import StateModeling as stm
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from Corona.LoadData import loadData, preprocessData
from Corona.CoronaModel import CoronaModel, plotTotalCases
from bokeh.io import push_notebook, show, output_notebook
from bokeh.models import Button
from ipywidgets import interact, interactive, interact_manual, widgets

output_notebook()

xpos = np.arange(0,100)/100.0

AllMeasured = loadData(r"COVID-19 Linelist 2020_04_23.xlsx", useThuringia = True, pullData=False)
# AllMeasured = preprocessData(AllMeasured)
# AllMeasured = loadData(useThuringia = False, pullData=False)
ExampleRegions = ['SK Jena', 'LK Greiz'] # 'SK Gera',
AllMeasured = preprocessData(AllMeasured, ReduceDistricts=ExampleRegions, SumDistricts=False, SumAges=True, SumGender=True)

M = CoronaModel(AllMeasured)

if False:
    mobdat = AllMeasured['mobility']
    mobdate = mobdat['date'].to_numpy()
    plt.figure('Retail and recreation');plt.plot(mobdat['retail_and_recreation_percent_change_from_baseline'].to_numpy())
    offsetDay=0; plt.xticks(range(offsetDay, len(mobdate), 7), [date for date in mobdate[offsetDay:-1:7]], rotation="vertical")
    plt.ylabel('Percent Change'); plt.tight_layout()

Tmax = 120

M.toFit(['r0', 'h', 'aT0', 'aBase', 'I0', 'd', 'rd', 'T0', 'q']) # 'q',
if AllMeasured['Cases'].shape[-2] > 1:
    M.toFit.append(['Age Border', 'Age Sigma'])

PopSum = np.sum(AllMeasured['Population'])
measured = AllMeasured['Cases'][:,  np.newaxis, :, :, :] / PopSum
measuredDead = AllMeasured['Dead'][:, np.newaxis, :, :, :] / PopSum
xlim = None  # (60,100)
# fittedVars, fittedRes = M.fit({'detected': measured}, Tmax, otype=otype, oparam=oparam, NIter=NIter, verbose=True, lossScale=lossScale)
FitDict = {'cases': measured}
if "Hospitalized" in AllMeasured.keys():
    FitDict['hospitalization'] = AllMeasured['Hospitalized'][:, np.newaxis, :, :, :]/ PopSum
FitDict['deaths'] = measuredDead

# SimDict = {'cases': None, 'cumul_cases': None, 'cumul_dead':None}

if True:
    otype = "L-BFGS"
    lossScale = 1.0  # 1e4
    oparam = {"normFac": 'max'}
else:
    lossScale = None
    otype = "nesterov"  # "adagrad"  "adadelta" "SGD" "nesterov"  "adam"
    learnrate = {"nesterov": 1000.0, "adam": 7e-7}
    oparam = {"learning_rate": tf.constant(learnrate[otype], dtype=stm.CalcFloatStr)}
# oparam['noiseModel'] = 'Poisson'
oparam['noiseModel'] = 'Gaussian'
# oparam['noiseModel'] = 'ScaledGaussian'  # is buggy? Why the NaNs?

# tf.config.experimental_run_functions_eagerly(True)

def doFit(NIter=0):
    fittedVars, fittedRes = M.fit(FitDict, Tmax, otype=otype, oparam=oparam, NIter=NIter, verbose=True, lossScale=lossScale)
    return fittedVars, fittedRes

fittedVars, fittedRes = doFit()
# simulated = M.simulate('simulated', FitDict, Tmax=Tmax)

def showSimRes(ymin=0.0001,ymax=1.0):
    ylim = [ymin,ymax]
    #p=M.showResultsBokeh(title=AllMeasured['Region'], Scale=PopSum, ylabel='occupancy', 
    #              xlim=xlim, dims=("District"), Dates=AllMeasured['Dates'], 
    #              legendPlacement='upper right', styles=['.','-','--'], ylim=ylim, figsize=[10,5])
    p=M.showResultsBokeh(title=AllMeasured['Region'], Scale=PopSum, ylabel='occupancy', 
                  xlim=xlim, dims=("District"), 
                  legendPlacement='upper right',ylim=ylim, figsize=[10,5])
    show(p, notebook_handle=True)
    return p

def showSimStates(ymin=1.0,ymax=1.0):
    ylim = [-ymin,ymax]
    M.showStatesBokeh(MinusOne=('S'), dims2d=None, Dates = AllMeasured['Dates'], ylim=ylim, figsize=[10,5])
    #M.assignNewVar('r0',r0)
    #M.assignNewVar('h',h)
    #simulated = M.simulate('simulated', FitDict, Tmax=Tmax)

# M.toFit(['r0', 'h', 'aT0', 'aBase', 'I0', 'd', 'rd', 'T0', 'q']) # 'q',

def assignParam(r0=None, h=0.04, aT0=6.6, aBase =0.14, I0=5e-9, d=0.13, rd=0.5, T0=12.4, q=0.0052):
    aDict = {'d':d, 'h':h, 'aT0':aT0, 'aBase':aBase, 'I0':10**(I0), 'rd':rd}
    for varN, newval in aDict.items():
        #'r0':r0, 'T0':T0, , 'q':q
        M.assignNewVar(varN, newval)
        print(varN+": "+str(newval))
    fittedVars, fittedRes = doFit()
    p=showSimRes()
    print('Loss: '+str(M.FitResultVars['Loss'].numpy())+'\n')



Loading BokehJS ...

unknown age.0... skipping ...
unknown age.0... skipping ...
unknown age.0... skipping ...
unknown age.0... skipping ...
unknown age.0... skipping ...
tracing doBuildModel
tracing traceModel
tracing time step 119
 .. done

Loss is: tf.Tensor(5.478647e-05, shape=(), dtype=float32)


In [3]:
#interact_manual(showSim,
#        ymin=ymin,
#        ymax=ymax)
# showSimRes(ymin=0.0001,ymax=1.0)

interact_manual(doFit,NIter=widgets.IntSlider(min=1, max=200, step=10, value=10));

def calcPlot():
    fittedVars, fittedRes = doFit()
    p=showSimRes()
    
#
d = interactive(lambda d: M.assignNewVar('d', d), d=(0,5,0.01))
h = interactive(lambda h: M.assignNewVar('h', h), h=(0,5,0.01))
aT0 = interactive(lambda aT0: M.assignNewVar('aT0', aT0), aT0=(0,5,0.01))
ui = widgets.HBox((d,h,aT0))
display(ui)

interact_manual(calcPlot,name="Show Plot");

#interact(assignParam, r0=(0,5,0.01), h=(0,5,0.01), aT0=(0,50,0.5), aBase =(0,1,0.01), I0=(-10,1,0.01), d=(0,1,0.01),
#         rd=(0,1,0.01), T0=(0,50,0.5), q=(0,1,0.01))
# push_notebook()

interactive(children=(IntSlider(value=10, description='NIter', max=200, min=1, step=10), Button(description='R…

interactive(children=(Button(description='Run Interact', style=ButtonStyle()), Output()), _dom_classes=('widge…

In [5]:
ylim = [0.0,10.0]
p = M.showResultsBokeh(title=AllMeasured['Region'], Scale=PopSum, ylabel='occupancy', 
                  xlim=xlim, dims=("District"),  
                  legendPlacement='upper right', ylim=ylim, figsize=[10,5])
show(p, notebook_handle=True)

In [3]:
M.getGUI()

IndexError: index 0 is out of bounds for axis 0 with size 0

In [14]:
def assignParam(r0=None, h=0.04, aT0=6.6, aBase =0.14, I0=5e-9, d=0.13, rd=0.5, T0=12.4, q=0.0052):
    aDict = {'d':d, 'h':h, 'aT0':aT0, 'aBase':aBase, 'I0':10**(I0), 'rd':rd}
    for varN, newval in aDict.items():
        #'r0':r0, 'T0':T0, , 'q':q
        M.assignNewVar(varN, newval)
        print(varN+": "+str(newval))
    fittedVars, fittedRes = doFit()
    p=showSimRes()
    print('Loss: '+str(M.FitResultVars['Loss'].numpy())+'\n')


In [17]:
from ipywidgets import interact, interact_manual, widgets


In [5]:
from bokeh.io import push_notebook, show, output_notebook

In [5]:
def showSimRes(ymin=0.0001,ymax=1.0):
    ylim = [ymin,ymax]
    #p=M.showResultsBokeh(title=AllMeasured['Region'], Scale=PopSum, ylabel='occupancy', 
    #              xlim=xlim, dims=("District"), Dates=AllMeasured['Dates'], 
    #              legendPlacement='upper right', styles=['.','-','--'], ylim=ylim, figsize=[10,5])
    p=M.showResultsBokeh(title=AllMeasured['Region'], Scale=PopSum, ylabel='occupancy', 
                  xlim=xlim, dims=("District"), 
                  legendPlacement='upper right',ylim=ylim, figsize=[10,5])
    show(p, notebook_handle=True)
    return p
